In [1]:

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import os
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings

/opt/anaconda3/envs/rag1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
print("Python path:", sys.executable)
print("\nPython version:", sys.version)

# Ver qué tienes instalado
import subprocess
result = subprocess.run([sys.executable, '-m', 'pip', 'list'], capture_output=True, text=True)
print("\nLibrerías instaladas:")
for line in result.stdout.split('\n'):
    if 'langchain' in line.lower():
        print(line)

Python path: /opt/anaconda3/envs/rag1/bin/python

Python version: 3.12.9 | packaged by conda-forge | (main, Mar  4 2025, 22:45:25) [Clang 18.1.8 ]

Librerías instaladas:
langchain                 0.3.27
langchain-classic         1.0.0
langchain-community       0.3.31
langchain-core            1.0.4
langchain-openai          0.3.35
langchain-pinecone        0.2.13
langchain-text-splitters  0.3.11


In [ ]:

# 1. Configurar credenciales
os.environ["PINECONE_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""

# 2. Conectar a Pinecone (aquí se conecta a la base de datos vectorial)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = PineconeVectorStore.from_existing_index(
    index_name="document-index",  # tu índice en Pinecone
    embedding=embeddings
)

# 3. Crear el retriever (esto es lo que consulta Pinecone)
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [5]:
# Inicializar el modelo
llm = ChatOpenAI(
    model="gpt-4o-mini",  # o "gpt-4o" si quieres el más potente
    temperature=0  # 0 para respuestas más determinísticas
)



# Crear el template del prompt
system_prompt = (
    "Eres un asistente experto que responde preguntas basándote "
    "únicamente en el contexto proporcionado. "
    "Si no encuentras la respuesta en el contexto, di que no tienes "
    "suficiente información.\n\n"
    "Contexto: {context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])



# Esta cadena toma los documentos recuperados y los procesa con el LLM
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Esta cadena conecta el retriever con el LLM
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [ ]:
response = rag_chain.invoke({"input": "¿cómo cargo imágenes a través de dropebox?"})

# Ver la respuesta
print("🤖 Respuesta:", response["answer"])
print("\n📄 Fuentes utilizadas:")
for i, doc in enumerate(response["context"], 1):
    print(f"\n--- Fuente {i} ---")
    print(doc.page_content[:200])

🤖 Respuesta: Para crear productos a través de una matriz, debes seguir estos pasos:

1. Recopila la información de los productos y descarga la tabla de equivalencias desde Productos -> Carga masiva -> Descargar tabla de equivalencias.
2. Completa la matriz de carga, asegurándote de que no haya celdas vacías, fórmulas o errores. La matriz debe contener columnas con los distintos campos/propiedades de los productos y el cuerpo debe incluir los valores correspondientes a cada SKU.
3. Asegúrate de trabajar en la pestaña "Productos", que debe estar ubicada en la primera posición a la izquierda. Las demás pestañas pueden servir como complemento o fuente de datos.
4. Una vez que la matriz esté completa, guarda el archivo en formato Excel (XLSX).
5. Ingresa a Productos -> Carga masiva -> "Selecciona Archivo" y selecciona el archivo que has creado.
6. En 'Carga tu excel con productos', selecciona el archivo y haz clic en Subir y previsualizar.
7. Luego de importar los productos, puedes proceder